In [1]:
# Imports
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F


from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset, TensorDataset

from torchsummary import summary
import torchvision as tv
import torchvision.transforms as T
import copy

import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline as backend_inline

backend_inline.set_matplotlib_formats("svg")


#### Pytorch device specific configuration ###

# Pytorch Gpu Configuration for Cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Pytorch Gpu Configuration for directml(AMD GPU)
# import torch_directml

# device = torch_directml.device()

# Set default device
torch.set_default_device(device)

In [2]:
# transformations
transform = T.Compose(
    [
        T.ToTensor(),
        T.RandomHorizontalFlip(p=0.5),
        T.Normalize(0.5, 0.5),
    ]
)

# import the data and simultaneously apply the transform
trainset = tv.datasets.FashionMNIST(
    root="../Datasets/", train=True, download=True, transform=transform
)
devtest = tv.datasets.FashionMNIST(
    root="../Datasets/", train=False, download=True, transform=transform
)

# split the devtest into two separate sets
randidx = np.random.permutation(10000)  # a random permutation of indices
devset = Subset(devtest, randidx[:6000])  # Subset for devset
testset = Subset(devtest, randidx[6000:])  # Subset for test

# transform to dataloaders
batchsize = 32
train_loader = DataLoader(trainset, batch_size=batchsize, shuffle=True, drop_last=True, generator=torch.Generator(device=device))
dev_loader = DataLoader(devset, batch_size=len(devset), generator=torch.Generator(device=device))
test_loader = DataLoader(testset, batch_size=len(testset), generator=torch.Generator(device=device))